# Pathological Gaits Recognition

## 0 - General Settings

In [ ]:
!pip install -q keras==3.8.0 tensorflow==2.18.0

import tensorflow as tf
import keras

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
SEED_VALUE=821
keras.utils.set_random_seed(SEED_VALUE)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import zipfile
import sys
import os
import time
import gc
import pickle

from google.colab import drive
from tensorflow.keras.utils import plot_model


In [ ]:
with zipfile.ZipFile("Utils.zip", "r") as zip_ref:
  zip_ref.extractall("Utils")
sys.path.append('/content/Utils')


In [ ]:
import Utils.Constants as _c

from Utils.Preprocessing import create_reference_df, create_dataset, create_dataset_AE, create_fusion_dataset

from Utils.LOSO_CV_Utils import (
    print_params,
    print_params_AE,
    remove_cache_from_drive,
    count_chunks,
    plot_history,
    print_time_info,
    compute_metrics,
    plot_history_AE,
    print_time_info_fus,
    plot_history_fus,
    compute_metrics_fus,
    plot_metrics_fus
)

from Utils.Analyze_Results import (
    merge_results,
    merge_loso_results,
    print_overall_time,
    print_overall_epochs,
    print_overall_cm,
    print_overall_clf_report,
    print_overall_accuracies,
    print_overall_history,
    print_overall_history_AE,
    print_overall_max_gpu_memory_usage,
    show_multiple_accuracies
)


In [ ]:
# connect to drive
drive.mount('/content/drive')


In [ ]:
data_dir = '' # data directory, e.g.: '/content/drive/MyDrive/Colab Notebooks/folder/data.zip'
local_data_dir = '' # local data directory, e.g.: '/content/data'

os.chdir('') # change to the directory where the notebook is located, e.g.: '/content/drive/MyDrive/Colab Notebooks/folder'


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

mpl.rcParams['figure.figsize'] = (10, 6)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['legend.fontsize'] = 'large'


## 1 - Create Data Folder and Reference df

In [ ]:
%%capture
!unzip "{data_dir}" -d "{local_data_dir}"


In [ ]:
reference_df = create_reference_df(local_data_dir, _c.convrt_gait_dict)

test_reference_df = reference_df[reference_df['subject']=='subject12']
loso_reference_df = reference_df[reference_df['subject']!='subject12']

print(f'{loso_reference_df.shape=}')
print(f'{test_reference_df.shape=}')
loso_reference_df.head()


In [ ]:
del reference_df

## 2 - Models Definition

In [ ]:
def build_gru(
  units:int,
  return_sequences:bool,
  bidirectional:bool,
  seed_value:int=SEED_VALUE,
  name:str=None
):
  gru = tf.keras.layers.GRU(units=units,
                            return_sequences=return_sequences,
                            kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value),
                            recurrent_initializer=tf.keras.initializers.Orthogonal(seed=seed_value),
                            bias_initializer=tf.keras.initializers.Zeros(),
                            name=name)

  return tf.keras.layers.Bidirectional(gru, name=name) if bidirectional else gru

def build_dense(
  units:int,
  activation:str,
  lambda_l2:float=None,
  seed_value:int=SEED_VALUE,
  name:str=None
):
  return tf.keras.layers.Dense(units=units, activation=activation,
                               kernel_regularizer=tf.keras.regularizers.l2(lambda_l2),
                               kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value),
                               bias_initializer=tf.keras.initializers.Zeros(),
                               name=name)

def build_conv(
  num_filters:int,
  kernel_size:tuple,
  activation:str,
  padding:str,
  seed_value:int=SEED_VALUE,
  name:str=None
):
  return tf.keras.layers.Conv2D(num_filters, kernel_size, activation=activation, padding=padding,
                                kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value),
                                name=name)

def build_AE_layer(
  units:int,
  name:str,
  layer_type:str
):
  if layer_type == 'gru':
    return tf.keras.layers.GRU(units=units, return_sequences=True, name=name)

  else: # layer_type == 'lstm'
    return tf.keras.layers.LSTM(units=units, return_sequences=True, name=name)


### 2.1 - Skeleton Models

#### 2.1.1 - GRU Model

In [ ]:
def GRU_Skeleton(input_shape:tuple, lambda_l2:float=None, bidirectional:bool=False):

  X_input = tf.keras.Input(input_shape, name='sk_input')

  # dense layer
  X = build_dense(units=input_shape[1], activation='relu', name='dense_input_sk')(X_input)

  # GRU layers
  X = build_gru(256, return_sequences=True, bidirectional=bidirectional, name='gru_sk0')(X_input)
  X = build_gru(256, return_sequences=True, bidirectional=bidirectional, name='gru_sk1')(X)
  X = build_gru(128, return_sequences=True, bidirectional=bidirectional, name='gru_sk2')(X)
  X = build_gru(128, return_sequences=False, bidirectional=bidirectional, name='gru_sk3')(X)

  # dense and regularization layers
  X = build_dense(units=100, lambda_l2=lambda_l2, activation=None, name='dense_sk0')(X)
  X = tf.keras.layers.Dropout(0.5)(X)
  X = tf.keras.layers.BatchNormalization(axis=-1, name='batchnorm_sk')(X)

  # dense layers with decreasing units
  for idx, units in enumerate([128, 64, 32]):
    X = build_dense(units=units, lambda_l2=lambda_l2, activation='relu', name=f'dense_sk{idx+1}')(X)

  # output layer
  X = build_dense(units=6, activation='softmax', name=f'dense_sk{idx+2}')(X)

  model = tf.keras.Model(inputs=X_input, outputs=X, name='SkeletonModel')
  return model


In [ ]:
# num_joints = 3*len(_c.joints_to_include)
num_joints = 3*len([0,1,2,3,4,11,18,19,20,21,22,23,24,25])
target_size = 50
input_shape = (target_size, num_joints)

sk_gru_exemple = GRU_Skeleton(input_shape)

sk_gru_exemple.summary()


#### 2.1.2 - Transformer Model

In [ ]:
# Positional Encoding Layer
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, max_len=100):
    super().__init__()
    self.max_len = max_len

  def build(self, input_shape):
    self.pos_embedding = self.add_weight(
      name="pos_embedding",
      shape=(self.max_len, input_shape[-1]),
      initializer="random_normal",
      trainable=True,
    )

  def call(self, x):
    length = tf.shape(x)[1]
    return x + self.pos_embedding[:length]

# Transformer Encoder Block
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0.1):

  attention_output = tf.keras.layers.MultiHeadAttention(
    key_dim=head_size, num_heads=num_heads, dropout=dropout
  )(inputs, inputs)

  attention_output = tf.keras.layers.Dropout(dropout)(attention_output)

  X = tf.keras.layers.Add()([inputs, attention_output])  # add resid

  # LayerNorm and Feed Forward
  X_norm = tf.keras.layers.LayerNormalization(epsilon=1e-6)(X)
  ff_output = tf.keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation=tf.keras.activations.gelu)(X_norm)
  ff_output = tf.keras.layers.Dropout(dropout)(ff_output)
  ff_output = tf.keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(ff_output)
  X = tf.keras.layers.Add()([X, ff_output])

  return X

# Final Transformer Model
def Transformer_Skeleton(
  input_shape,
  head_size=128,
  num_heads=4,
  ff_dim=256,
  num_transformer_blocks=6,
  mlp_units=[128, 64, 32],
  lambda_l2=5e-3,
  dropout=0.3,
  mlp_dropout=0.2,
):
  inputs = keras.Input(shape=input_shape)
  X = PositionalEncoding()(inputs)

  for _ in range(num_transformer_blocks):
    X = transformer_encoder(X, head_size, num_heads, ff_dim, dropout)

  X = tf.keras.layers.GlobalAveragePooling1D(name='avg_pool_sk')(X)

  for idx, units in enumerate(mlp_units):
    X = build_dense(units, lambda_l2=lambda_l2, activation='gelu', name=f'dense_sk{idx+1}')(X)
    X = tf.keras.layers.Dropout(mlp_dropout)(X)

  outputs = tf.keras.layers.Dense(6, activation='softmax', name=f'dense_sk{len(mlp_units)+1}')(X)

  model = keras.Model(inputs, outputs)
  return model


In [ ]:
sk_trn_exemple = Transformer_Skeleton(input_shape)
plot_model(sk_trn_exemple, show_shapes=True, show_layer_names=True, dpi=100)


In [ ]:
total_params = sk_trn_exemple.count_params()
trainable_params = sum([tf.keras.backend.count_params(p) for p in sk_trn_exemple.trainable_weights])
non_trainable_params = sum([tf.keras.backend.count_params(p) for p in sk_trn_exemple.non_trainable_weights])

print(f"Total params: {total_params:,} ({total_params * 4 / (1024 ** 2):.2f} MB)")
print(f"Trainable params: {trainable_params:,} ({trainable_params * 4 / (1024 ** 2):.2f} MB)")
print(f"Non-trainable params: {non_trainable_params:,} ({non_trainable_params * 4 / (1024 ** 2):.2f} B)")


#### 2.1.3 - Autoencoder

In [ ]:
def build_encoder(enc_input_shape:tuple, code_size:int, layer_type:str='lstm'):

  num_units = enc_input_shape[1]

  X_input = tf.keras.Input(tuple(enc_input_shape), name='sk_input')

  # GRU layers
  X = build_AE_layer(num_units, name='enc_rnn_sk0', layer_type=layer_type)(X_input)
  X = build_AE_layer(num_units, name='enc_rnn_sk1', layer_type=layer_type)(X)
  X = build_AE_layer(num_units, name='enc_rnn_sk2', layer_type=layer_type)(X)
  X = build_AE_layer(code_size, name='enc_rnn_sk3', layer_type=layer_type)(X)

  encoder_model = tf.keras.Model(inputs=X_input, outputs=X, name="Encoder_Skeleton")

  return encoder_model

def build_decoder(dec_input_shape:tuple, feature_dim:int, layer_type:str='lstm'):

  decoder_input = tf.keras.Input(dec_input_shape, name='decoder_input')

  X = build_AE_layer(feature_dim, name='dec_rnn_sk0', layer_type=layer_type)(decoder_input)

  decoder_model = tf.keras.Model(inputs=decoder_input, outputs=X, name="Decoder_Skeleton")

  return decoder_model

def Autoencoder_Skeleton(input_shape:tuple, code_size:int=65, layer_type:str='lstm'):
  # define encoder
  encoder_model = build_encoder(input_shape, code_size, layer_type)

  # get dimensions
  input_shape_dec = encoder_model.output_shape[1:]
  feature_dim_dec = input_shape[-1]

  # define decoder
  decoder_model = build_decoder(input_shape_dec, feature_dim_dec, layer_type)

  # connect encoder and decoder
  inputs = encoder_model.input
  encoded_seq = encoder_model(inputs)
  outputs = decoder_model(encoded_seq)

  autoencoder = tf.keras.Model(inputs=inputs, outputs=outputs, name="RNN_Autoencoder")

  return autoencoder, encoder_model, decoder_model


In [ ]:
sk_AE_model, sk_enc_model, sk_dec_model = Autoencoder_Skeleton(input_shape)
sk_enc_model.summary()


In [ ]:
sk_dec_model.summary()


In [ ]:
sk_AE_model.summary()

### 2.2 - Speed Model

In [ ]:
def GRU_Speed(input_shape:tuple, lambda_l2:float=0, bidirectional:bool=False):

  X_input = tf.keras.Input(input_shape, name='sp_input')

  # dense layer
  X = build_dense(units=input_shape[1], activation='relu', name='dense_input_sp')(X_input)

  # GRU layers
  X =  build_gru(128, return_sequences=True, bidirectional=bidirectional, name="gru_sp0")(X)
  X =  build_gru(128, return_sequences=True, bidirectional=bidirectional, name="gru_sp1")(X)
  X =  build_gru(64, return_sequences=True, bidirectional=bidirectional, name="gru_sp2")(X)
  X =  build_gru(64, return_sequences=False, bidirectional=bidirectional, name="gru_sp3")(X)

  # dense and regularization layers
  X = build_dense(units=50, lambda_l2=None, activation='relu', name='dense_sp0')(X)
  X = tf.keras.layers.Dropout(0.5)(X)
  X = tf.keras.layers.BatchNormalization(axis=-1, name='batchnorm_sp')(X)

  # dense layers with decreasing units
  for idx, units in enumerate([64, 32, 16]):
    X = build_dense(units=units, lambda_l2=lambda_l2, activation='relu', name=f'dense_sp{idx+1}')(X)

  # output layer
  X = build_dense(units=6, activation='softmax', name=f'dense_sp{idx+2}')(X)

  model = tf.keras.Model(inputs=X_input, outputs=X,  name='SpeedModel')

  return model


In [ ]:
num_joints = len(_c.joints_to_include)
target_size = 50
input_shape = (target_size, num_joints)

sp_gru_exemple = GRU_Speed(input_shape)

sp_gru_exemple.summary()

### 2.3 - Foot Pressure Model

In [ ]:
def CNN_Foot(input_shape=(128, 48, 1), lambda_l2:float=0.0):

  X_input = tf.keras.Input(input_shape, name='ft_input')

  # Conv Block 1
  X = build_conv(16, (3, 3), activation='relu', padding='same', name='conv_ft0')(X_input)
  X = build_conv(16, (3, 3), activation='relu', padding='same', name='conv_ft1')(X)
  X = tf.keras.layers.MaxPooling2D((2, 2), name='pool_ft1')(X)

  # Conv Block 2
  X = build_conv(32, (3, 3), activation='relu', padding='same', name='conv_ft2')(X)
  X = build_conv(32, (3, 3), activation='relu', padding='same', name='conv_ft3')(X)
  X = tf.keras.layers.MaxPooling2D((2, 2), name='pool_ft2')(X)

  # Conv Block 3
  X = build_conv(64, (3, 3), activation='relu', padding='same', name='conv_ft4')(X)
  X = build_conv(64, (3, 3), activation='relu', padding='same', name='conv_ft5')(X)
  X = tf.keras.layers.MaxPooling2D((2, 2), name='pool_ft3')(X)

  # Conv Block 4
  X = build_conv(128, (3, 3), activation='relu', padding='same', name='conv_ft6')(X)
  X = tf.keras.layers.MaxPooling2D((2, 2), name='pool_ft4')(X)

  # Flatten + Projection to feature vector
  X = tf.keras.layers.Flatten(name='flatten_ft')(X)
  X = build_dense(units=512, activation='relu', name='dense_ft0')(X)
  X = build_dense(units=256, activation='relu', name='dense_ft1')(X)
  X = build_dense(units=128, activation='relu', name='dense_ft2')(X)
  X = tf.keras.layers.Dropout(0.5, name='dropout_ft')(X)
  X = tf.keras.layers.BatchNormalization(axis=-1, name='batchn_ft')(X)

  # FC layers
  for idx, units in enumerate([128, 64, 32]):
    X = build_dense(units=units, lambda_l2=lambda_l2, activation='relu', name=f'dense_ft{idx+3}')(X)

  X = build_dense(units=6, activation='softmax', name=f'dense_ft{idx+4}')(X)

  # Build the model
  model = tf.keras.Model(inputs=X_input, outputs=X, name='FooTModel')

  return model


In [ ]:
input_shape = (128, 48, 1)

ft_cnn_exemple = CNN_Foot(input_shape)
ft_cnn_exemple.summary()


## 3 - Leave-One-Subject-Out Cross-Validation (LOSO CV)



We will define two functions, **define_model** that build the model based on datat type and some other parameters, and **LOSO_CV** that performs Leave-One-Subject-Out Cross-Validation for training and evaluating our models on skeleton, foot or speed data. The idea is to **train on all subjects except one**, and **test on the left-out subject**, rotating through all subjects.



### 3.1 - Single Classifier

In [ ]:
def define_model(
  data_type:str,
  model_type:str,
  target_size:int=50,
  num_features:int=42,
  img_size:tuple=(128,48,1),
  lambda_l2:float=0.0,
  bidirectional:bool=False,
):

  if data_type=='skeleton':
    SKELETON_INPUT_SHAPE = (target_size, num_features)

    if model_type=='gru':
      model = GRU_Skeleton(input_shape=SKELETON_INPUT_SHAPE,
                           lambda_l2=lambda_l2,
                           bidirectional=bidirectional)

    else: # model_type=='transformer':
      model = Transformer_Skeleton(input_shape=SKELETON_INPUT_SHAPE)

  elif data_type=='speed':
    SPEED_INPUT_SHAPE = (target_size, num_features)
    model = GRU_Speed(input_shape=SPEED_INPUT_SHAPE,
                      lambda_l2=lambda_l2,
                      bidirectional=bidirectional)

  else: # data_type=='foot'
    model = CNN_Foot(input_shape=img_size, lambda_l2=lambda_l2)

  return model


**Step-by-Step guide on how LOSO_CV works**:

1. **Subject Loop (LOSO)**
   For each subject:

   * That subject is used **exclusively for validation**, while all others are used for training.
   * Augmented samples (like flipped ones) are **excluded from validation**.

2. **Dataset Creation**

   * Training and validation datasets are created with proper preprocessing (e.g.: normalization, cropping, joint selection, ecc).

4. **Train Classifier**

   * Trains the classifier model.
   * Evaluates the model on the left-out subject and computes:
     * Time informations
     * RAM informations
     * Confusion Matrix
     * Classification Report
     * Accuracy

5. **Cleanup**

   * After each subject, it cleans up memory and cached datasets.
   * Clears TensorFlow backend session.

6. **Returns Results**

    * A dictionary with:

      * Training histories for AE and classifier
      * Time taken per subject
      * RAM usage
      * Confusion matrices
      * Classification reports


In [ ]:
print(tf.config.list_physical_devices('GPU'))


In [ ]:
# https://stackoverflow.com/questions/70763324/how-to-print-the-maximum-memory-used-during-kerass-model-fit
class MemoryLoggingCallback(tf.keras.callbacks.Callback):
  def __init__(self):
    super().__init__()
    self.memory_log = {}  # store memory usage per epoch

  def on_epoch_end(self, epoch, logs=None):
    gpu_dict = tf.config.experimental.get_memory_info('GPU:0')
    current_gb = float(gpu_dict['current']) / (1024 ** 3)
    peak_gb = float(gpu_dict['peak']) / (1024 ** 3)

    self.memory_log[epoch] = {'current_gb': current_gb,
                              'peak_gb': peak_gb}

    # tf.print('\nGPU memory details [current: {:.2f} GB, peak: {:.2f} GB]'.format(current_gb, peak_gb))


In [ ]:
def LOSO_CV(
  reference_df:pd.DataFrame=None,
  test_reference_df:pd.DataFrame=None,
  select_x_subj:list=None,
  data_type:str='skeleton',
  model_type:str='gru',
  joints:list=list(range(0,32)),
  target_size:int=50,
  layer_type:str='lstm',
  img_size:tuple=(128,48,1),
  clean_data:bool=True,
  norm:bool=True,
  crop_type:str='aggressive_center',
  center_ft:bool=True,
  bidirectional:bool=False,
  batch_size:int=30,
  num_epochs_clf:int=200,
  patience_clf:int=30,
  lambda_l2:float=1e-2,
  lr:float=1e-4,
  data_dir:str=None,
):

  print_params(**locals())
  dict_history, dict_times, dict_memory = {}, {}, {}
  dict_cm, dict_clf_rep, dict_acc = {}, {}, {}
  if test_reference_df is not None:
    test_results = {'cm':{}, 'clf_rep':{}, 'acc':{}}

  sorted_subjects = sorted(reference_df['subject'].unique(), key=lambda x: int(x.replace('subject', '')))
  if isinstance(select_x_subj, list):
    sorted_subjects = [sorted_subjects[i-1] for i in select_x_subj]

  for subject in sorted_subjects:

    print(f'\n=== {subject.upper()} ===\n')

    # create train and validation reference data
    train_reference_df, valid_reference_df = reference_df[reference_df['subject']!=subject], reference_df[reference_df['subject']==subject]
    valid_reference_df = valid_reference_df[~valid_reference_df.index.str.contains('flipped')] # no augmented data for the validation set
    if test_reference_df is not None:
      test_reference_df = test_reference_df[~test_reference_df.index.str.contains('flipped')] # no augmented data for the test set # ADD

    if crop_type=='split_subsequence':
      chunk_counts = [count_chunks(fp, joints, clean_data=True, norm=True, target_size=target_size) for fp in train_reference_df.index]
      train_steps = int(np.ceil(sum(chunk_counts)/batch_size))
    else:
      train_steps = int(np.ceil(len(train_reference_df)/batch_size))

    valid_steps = int(np.ceil(len(valid_reference_df)/batch_size))
    if test_reference_df is not None:
      test_steps = int(np.ceil(len(test_reference_df)/batch_size))# ADD

    # common args to create train and validation sets
    common_args = {'data_type':data_type,
                   'joints':joints,
                   'target_size':target_size,
                   'img_size':img_size,
                   'clean_data':clean_data,
                   'norm':norm,
                   'center_ft':center_ft,
                   'batch_size':batch_size}

    train_data = create_dataset(**common_args,
                                reference_df=train_reference_df,
                                shuffle=True, crop_type=crop_type,
                                cache_file=f'train_{data_type}_{subject}')

    crop_type_valid = 'aggressive_center' if crop_type=='split_subsequence' else crop_type
    valid_data = create_dataset(**common_args,
                                reference_df=valid_reference_df,
                                shuffle=False, crop_type=crop_type_valid,
                                cache_file=f'valid_{data_type}_{subject}')

    if test_reference_df is not None:
        test_data = create_dataset(**common_args,
                                   reference_df=test_reference_df,
                                   shuffle=False, crop_type=crop_type_valid,
                                   cache_file=f'test_{data_type}_{subject}')

    num_features = len(joints) * 3 if data_type == 'skeleton' else len(joints) if data_type == 'speed' else 0
    # initialize classifier
    model_clf = define_model(data_type=data_type, model_type=model_type,
                             target_size=target_size, num_features=num_features,
                             img_size=img_size, lambda_l2=lambda_l2,bidirectional=bidirectional)

    model_clf.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=lr),
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                      metrics=['accuracy'])

    early_stop_callback_clf = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience_clf, restore_best_weights=True)
    memory_callback_clf = MemoryLoggingCallback()

    start_time_clf = time.time()
    history = model_clf.fit(train_data,
                            epochs=num_epochs_clf,
                            steps_per_epoch=train_steps,
                            validation_data=valid_data,
                            validation_steps=valid_steps,
                            callbacks=[early_stop_callback_clf, memory_callback_clf],
                            verbose=0)

    delta_time_clf = time.time() - start_time_clf

    # show results
    print_time_info(subject, history, delta_time_clf)
    plot_history(history, subject)
    cm, df_report, acc = compute_metrics(model_clf, valid_reference_df, valid_data, valid_steps, subject)
    if test_reference_df is not None:
      cm_test, df_report_test, acc_test = compute_metrics(model_clf, test_reference_df, test_data, test_steps, 'subject12')
    print('\n')

    dict_times[subject], dict_history[subject], dict_memory[subject] = delta_time_clf, history, memory_callback_clf.memory_log
    dict_cm[subject], dict_clf_rep[subject], dict_acc[subject] = cm, df_report, acc
    if test_reference_df is not None:
      test_results['cm'][subject], test_results['clf_rep'][subject], test_results['acc'][subject] = cm_test, df_report_test, acc_test

    # remove cache and clear session
    del train_data, valid_data, model_clf, history, early_stop_callback_clf, memory_callback_clf
    remove_cache_from_drive(cache_data=f'train_{data_type}_{subject}', data_dir=data_dir)
    remove_cache_from_drive(cache_data=f'valid_{data_type}_{subject}', data_dir=data_dir)
    if test_reference_df is not None:
      remove_cache_from_drive(cache_data=f'test_{data_type}_{subject}', data_dir=data_dir)
    tf.keras.backend.clear_session()
    gc.collect()

  # ensemble reuslts
  results = {
      'dict_memory':dict_memory,
      'dict_history':dict_history,
      'dict_times':dict_times,
      'dict_cm':dict_cm,
      'dict_clf_rep':dict_clf_rep,
      'dict_acc':dict_acc
      }
  if test_reference_df is not None:
    results['test_results'] = test_results

  return results


### 3.2 - AE + Classifier

Basically the idea is to train an autoencoder using all joints (features) and let it extract the relevant informations, then pass the extracted features through the classifier.

We define a new LOSO CV function and a new function to choose the model (between classifier and AE).

In [ ]:
def define_model_AE(
  model_type:str,
  layer_type:str=None,
  num_features:int=42,
  code_size:int=None,
):

  if model_type=='AE':
    SKELETON_INPUT_SHAPE = (50, num_features) # length of the sequence + #joints
    AE, Enc, Dec = Autoencoder_Skeleton(input_shape=SKELETON_INPUT_SHAPE,
                                        code_size=code_size,
                                        layer_type=layer_type)
    return AE, Enc, Dec

  elif model_type=='clf':
    SKELETON_INPUT_SHAPE = (50, code_size) # length of the sequence + #extracted features

    model = GRU_Skeleton(input_shape=SKELETON_INPUT_SHAPE,
                         lambda_l2=5e-3,
                         bidirectional=False)

    return model


In [ ]:
def LOSO_CV_AE(
  reference_df:pd.DataFrame=None,
  select_x_subj:list=None,
  code_size:int=50,
  layer_type:str='lstm',
  batch_size:int=30,
  num_epochs_clf:int=200,
  patience_clf:int=30,
  lr:float=1e-4,
  num_epochs_AE:int=5000,
  patience_AE:int=10,
  data_dir:str=None,
):

  assert layer_type in ['lstm', 'gru'], f"layer_type must be either 'lstm' or 'gru' but is set to: {layer_type}"

  print_params_AE(**locals())

  dict_history_clf, dict_times_clf, dict_memory_clf = {}, {}, {}
  dict_cm_clf, dict_clf_rep_clf, dict_acc_clf = {}, {}, {}
  dict_history_AE, dict_times_AE, dict_memory_AE = {}, {}, {}

  # AE only for skeleton data
  data_type = 'skeleton'

  sorted_subjects = sorted(reference_df['subject'].unique(), key=lambda x: int(x.replace('subject', '')))
  if isinstance(select_x_subj, list):
    sorted_subjects = [sorted_subjects[i-1] for i in select_x_subj]

  for subject in sorted_subjects:

    print(f'\n=== {subject.upper()} ===\n')

    # create train and validation reference data
    train_reference_df, valid_reference_df = reference_df[reference_df['subject']!=subject], reference_df[reference_df['subject']==subject]
    valid_reference_df = valid_reference_df[~valid_reference_df.index.str.contains('flipped')] # no augmented data for the validation set

    train_steps = int(np.ceil(len(train_reference_df)/batch_size))
    valid_steps = int(np.ceil(len(valid_reference_df)/batch_size))

    # 1) We need to create the dataset to train AE
    train_data_AE = create_dataset_AE(reference_df=train_reference_df, train_or_valid='train',
                                      train_AE=True, batch_size=batch_size, shuffle=True,
                                      cache_file=f'train_AE_{data_type}_{subject}')

    valid_data_AE = create_dataset_AE(reference_df=valid_reference_df, train_or_valid='valid',
                                      train_AE=True, batch_size=batch_size, shuffle=False,
                                      cache_file=f'valid_AE_{data_type}_{subject}')

    # 2) initialize and train AE
    num_features = 3*len(list(range(0,32)))
    model_AE, model_encoder, model_decoder = define_model_AE(model_type='AE', layer_type=layer_type, num_features=num_features, code_size=code_size)

    model_AE.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=lr),
                     loss=tf.keras.losses.MeanSquaredError(),
                     metrics=['mse'])

    early_stop_callback_AE = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience_clf, restore_best_weights=True)
    memory_callback_AE = MemoryLoggingCallback()

    start_time_AE = time.time()
    history_AE = model_AE.fit(train_data_AE,
                              epochs=num_epochs_AE,
                              steps_per_epoch=train_steps,
                              validation_data=valid_data_AE,
                              validation_steps=valid_steps,
                              callbacks=[early_stop_callback_AE, memory_callback_AE],
                              verbose=0)

    delta_time_AE = time.time() - start_time_AE

    # 3) show AE results
    print_time_info(subject, history_AE, delta_time_AE)
    plot_history_AE(history_AE, subject)
    print('\n')

    # 4) create dataset for the classifier (extract features using AE)
    train_data_clf = create_dataset_AE(reference_df=train_reference_df, train_or_valid='train',
                                       train_AE=False, encoder_model=model_encoder,
                                       batch_size=batch_size, shuffle=True,
                                       cache_file=f'train_clf_{data_type}_{subject}')

    valid_data_clf = create_dataset_AE(reference_df=valid_reference_df, train_or_valid='valid',
                                       train_AE=False, encoder_model=model_encoder,
                                       batch_size=batch_size, shuffle=False,
                                       cache_file=f'valid_clf_{data_type}_{subject}')

    # 5) initialize and train classifier
    model_clf = define_model_AE(model_type='clf', code_size=code_size)

    model_clf.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=lr),
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                      metrics=['accuracy'])

    early_stop_callback_clf = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience_clf, restore_best_weights=True)
    memory_callback_clf = MemoryLoggingCallback()

    start_time_clf = time.time()
    history_clf = model_clf.fit(train_data_clf,
                                epochs=num_epochs_clf,
                                steps_per_epoch=train_steps,
                                validation_data=valid_data_clf,
                                validation_steps=valid_steps,
                                callbacks=[early_stop_callback_clf, memory_callback_clf],
                                verbose=0)

    delta_time_clf = time.time() - start_time_clf

    # 6) show classifier results
    print_time_info(subject, history_clf, delta_time_clf)
    plot_history(history_clf, subject)
    cm, df_report, acc = compute_metrics(model_clf, valid_reference_df, valid_data_clf, valid_steps, subject)
    print('\n')

    # save all results
    dict_times_clf[subject], dict_history_clf[subject], dict_memory_clf[subject] = delta_time_clf, history_clf, memory_callback_clf.memory_log
    dict_cm_clf[subject], dict_clf_rep_clf[subject], dict_acc_clf[subject] = cm, df_report, acc
    dict_history_AE[subject], dict_times_AE[subject], dict_memory_AE[subject] = history_AE, delta_time_AE, memory_callback_AE.memory_log

    # remove cache and clear session
    del train_data_clf, valid_data_clf, train_data_AE, valid_data_AE
    del model_AE, model_encoder, model_decoder, early_stop_callback_AE, memory_callback_AE
    del model_clf, history_clf, early_stop_callback_clf, memory_callback_clf

    remove_cache_from_drive(cache_data=f'train_AE_{data_type}_{subject}', data_dir=data_dir)
    remove_cache_from_drive(cache_data=f'valid_AE_{data_type}_{subject}', data_dir=data_dir)
    remove_cache_from_drive(cache_data=f'train_clf_{data_type}_{subject}', data_dir=data_dir)
    remove_cache_from_drive(cache_data=f'valid_clf_{data_type}_{subject}', data_dir=data_dir)

    tf.keras.backend.clear_session()
    gc.collect()

  # ensemble results
  results = {
      'dict_memory':dict_memory_clf,
      'dict_history':dict_history_clf,
      'dict_times':dict_times_clf,
      'dict_cm':dict_cm_clf,
      'dict_clf_rep':dict_clf_rep_clf,
      'dict_acc':dict_acc_clf,
      'dict_history_AE':dict_history_AE,
      'dict_times_AE':dict_times_AE,
      'dict_memory_AE':dict_memory_AE
      }

  return results


## 4 - Skeleton LOSO CV

### 4.1 - GRU Classifier


| Crop Type           | #Frames | Avg. Accuracy |
|-----------------------|---------|----------------|
| top                   | 100     | 0.4515         |
| bottom                | 100     | 0.8712         |
| center                | 100     | 0.8129         |
| random                | 100     | 0.7538         |
| 40perc                | 100     | 0.8879         |
| aggressive_center     | 50      | 0.8621         |
| aggressive_random     | 50      | 0.8780         |
| split_subsequence     | 50      | 0.8924         |


In [ ]:
our_params = {'reference_df': loso_reference_df,
              'model_type': 'gru',
              'data_type': 'skeleton',
              'target_size': 50,
              'joints': _c.joints_to_include,
              'clean_data': True,
              'norm': True,
              'crop_type': 'split_subsequence',
              'batch_size': 30,
              'bidirectional': False,
              'lambda_l2': 5e-3,
              'lr': 1e-4,
              'num_epochs_clf': 200,
              'patience_clf': 30,
              'data_dir': data_dir,
              }

results = LOSO_CV(**our_params)


In [ ]:
print_overall_time(results)


In [ ]:
print_overall_history(results)


In [ ]:
print_overall_epochs(results)


In [ ]:
print_overall_cm(results)


In [ ]:
print_overall_clf_report(results)


In [ ]:
print_overall_accuracies(results)


In [ ]:
print_overall_max_gpu_memory_usage(results)


### 4.2 - Bidirectional GRU Classifier

The best crop strategy was selected and the same architecture as the previous section but using bidirectional GRU is used to perform LOSO-CV

In [ ]:
our_params = {'reference_df': loso_reference_df,
              'model_type': 'gru',
              'data_type': 'skeleton',
              'target_size': 50,
              'joints': _c.joints_to_include,
              'clean_data': True,
              'norm': True,
              'crop_type': 'split_subsequence',
              'batch_size': 30,
              'bidirectional': True,
              'lambda_l2': 5e-3,
              'lr': 1e-4,
              'num_epochs_clf': 200,
              'patience_clf': 30,
              'data_dir': data_dir,
              }

results = LOSO_CV(**our_params)


In [ ]:
print_overall_time(results)


In [ ]:
print_overall_history(results)


In [ ]:
print_overall_epochs(results)


In [ ]:
print_overall_cm(results)


In [ ]:
print_overall_clf_report(results)


In [ ]:
print_overall_accuracies(results)


In [ ]:
print_overall_max_gpu_memory_usage(results)


### 4.3 - Transformer Classifier


In [ ]:
our_params = {'reference_df': loso_reference_df,
              'model_type': 'transformer',
              'data_type': 'skeleton',
              'target_size': 50,
              'joints': _c.joints_to_include,
              'clean_data': True,
              'norm': True,
              'crop_type': 'split_subsequence',
              'batch_size': 30,
              'lambda_l2': 5e-3,
              'lr': 1e-4,
              'num_epochs_clf': 200,
              'patience_clf': 30,
              'data_dir':data_dir,
              }

results = LOSO_CV(**our_params)


In [ ]:
print_overall_time(results)


In [ ]:
print_overall_history(results)


In [ ]:
print_overall_epochs(results)


In [ ]:
print_overall_cm(results)


In [ ]:
print_overall_clf_report(results)


In [ ]:
print_overall_accuracies(results)


In [ ]:
print_overall_max_gpu_memory_usage(results)


### 4.4 - Autoencoder + Classifier

| Code Size | Avg Accuracy |
|-----------|--------------|
| 50 | 0.8515 |
| 60 | 0.8818 |
| 70 | 0.8481 |
| 80 | 0.8368 |


In [ ]:
our_params = {'reference_df': loso_reference_df,
              'select_x_subj': [1,2,3,4,5,6],
              'layer_type': 'lstm',
              'code_size': 60,
              'batch_size': 30,
              'num_epochs_clf': 200,
              'patience_clf': 30,
              'num_epochs_AE': 5000,
              'patience_AE': 10,
              'lr': 1e-4,
              'data_dir': data_dir,
              }

results1 = LOSO_CV_AE(**our_params)


In [ ]:
# to save results
# with open('SK_LOSO_LSTM_AE50a.pkl', 'wb') as f:
#   pickle.dump(results1, f)


In [ ]:
our_params = {'reference_df': loso_reference_df,
              'select_x_subj': [7,8,9,10,11],
              'layer_type': 'lstm',
              'code_size': 60,
              'batch_size': 30,
              'num_epochs_clf': 200,
              'patience_clf': 30,
              'num_epochs_AE': 5000,
              'patience_AE': 10,
              'lr': 1e-4,
              'data_dir': data_dir,
              }

results2 = LOSO_CV_AE(**our_params)


In [ ]:
# # to save results
# with open('SK_LOSO_LSTM_AE50b.pkl', 'wb') as f:
#   pickle.dump(results2, f)


In [ ]:
# to load results
# with open('SK_LOSO_LSTM_AE50a.pkl', 'rb') as f:
#   results1 = pickle.load(f)

# with open('SK_LOSO_LSTM_AE50b.pkl', 'rb') as f:
#   results2 = pickle.load(f)

results = merge_results(results1, results2)


In [ ]:
print_overall_time(results, clf_or_ae='AE')


In [ ]:
print_overall_time(results)


In [ ]:
print_overall_history_AE(results)


In [ ]:
print_overall_history(results)


In [ ]:
print_overall_cm(results)


In [ ]:
print_overall_clf_report(results)


## 5 - Speed LOSO CV

In [ ]:
our_params = {'reference_df': loso_reference_df,
              'model_type': 'gru',
              'data_type': 'speed',
              'target_size': 50,
              'joints': _c.joints_to_include,
              'clean_data': True,
              'norm': True,
              'crop_type': 'split_subsequence',
              'batch_size': 30,
              'lambda_l2': 5e-3,
              'lr': 1e-4,
              'num_epochs_clf': 200,
              'patience_clf': 30,
              'data_dir': data_dir,
              }

results = LOSO_CV(**our_params)


In [ ]:
print_overall_time(results)


In [ ]:
print_overall_history(results)


In [ ]:
print_overall_epochs(results)


In [ ]:
print_overall_cm(results)


In [ ]:
print_overall_clf_report(results)


In [ ]:
print_overall_accuracies(results)


In [ ]:
print_overall_max_gpu_memory_usage(results)


## 6 - Foot Pressure LOSO CV

In [ ]:
our_params = {'reference_df': loso_reference_df,
              'data_type': 'foot',
              'img_size': (128,48,1),
              'norm': True,
              'center_ft': True,
              'batch_size': 30,
              'lambda_l2': 5e-3,
              'lr': 5e-5,
              'num_epochs_clf': 200,
              'patience_clf': 30,
              'data_dir':data_dir,
              }

results = LOSO_CV(**our_params)


In [ ]:
print_overall_time(results)


In [ ]:
print_overall_history(results)


In [ ]:
print_overall_epochs(results)


In [ ]:
print_overall_cm(results)


In [ ]:
print_overall_clf_report(results)


In [ ]:
print_overall_accuracies(results)


In [ ]:
print_overall_max_gpu_memory_usage(results)


## 7 - Fusion Model LOSO CV

In this section we build the fusion model that takes as input skeleton, foot and speed data. The three data types are passed through the best classifier among the one of the previous sections, the classifiers are then cut before the batch normalization layer such that the features extracted from each one are concatenated and passed through a final MLP.

4 configurations of fusion models were tried:
1. All data types (Skeleton + Foot + Speed)
2. Skeleton + Foot
3. Skeleton + Speed
4. Speed + Foot

### 7.1 - Define Model

In [ ]:
def build_fusion_model(model_dict, data_types_to_include, freeze_base_models:bool=True, lambda_l2_fus:float=5e-3):
  inputs = []
  outputs = []

  if 'skeleton' in data_types_to_include:
    sk_model = model_dict['skeleton']
    sk_model.trainable = not freeze_base_models
    sk_trunc = tf.keras.Model(inputs=sk_model.input,
                              outputs=sk_model.get_layer(f'dense_sk0').output)
                              #outputs=sk_model.get_layer(f'avg_pool_sk').output)

    inputs.append(sk_trunc.input)
    outputs.append(sk_trunc.output)

  if 'speed' in data_types_to_include:
    sp_model = model_dict['speed']
    sp_model.trainable = not freeze_base_models
    sp_trunc = tf.keras.Model(inputs=sp_model.input,
                              outputs=sp_model.get_layer(f'dense_sp0').output)
    inputs.append(sp_trunc.input)
    outputs.append(sp_trunc.output)

  if 'foot' in data_types_to_include:
    ft_model = model_dict['foot']
    ft_model.trainable = not freeze_base_models
    ft_trunc = tf.keras.Model(inputs=ft_model.input,
                              outputs=ft_model.get_layer(f'dense_ft2').output)
    inputs.append(ft_trunc.input)
    outputs.append(ft_trunc.output)

  # concatenate selected features
  X = tf.keras.layers.Concatenate(name='concat_features')(outputs)

  # FC layers
  X = tf.keras.layers.Dropout(0.3, name='dropout_fus')(X)
  X = tf.keras.layers.BatchNormalization(axis=-1, name='batchnorm_fus')(X)

  for idx, units in enumerate([128, 64, 32]):
    X = build_dense(units=units, lambda_l2=lambda_l2_fus, activation='relu', name=f'dense_fus{idx}')(X)
  X = build_dense(units=6, activation='softmax', name=f'dense_fus{idx+1}')(X)

  fusion_model = tf.keras.Model(inputs=inputs, outputs=X, name='FusionModel')
  return fusion_model


In [ ]:
model_dict = {'skeleton': sk_gru_exemple,
              'speed': sp_gru_exemple,
              'foot': ft_cnn_exemple}

data_types_to_include = ['skeleton', 'speed', 'foot']
fus_model_exemple = build_fusion_model(model_dict, data_types_to_include)

plot_model(fus_model_exemple, show_shapes=True, show_layer_names=True, dpi=100)


In [ ]:
total_params = fus_model_exemple.count_params()
trainable_params = sum([tf.keras.backend.count_params(p) for p in fus_model_exemple.trainable_weights])
non_trainable_params = sum([tf.keras.backend.count_params(p) for p in fus_model_exemple.non_trainable_weights])

print(f"Total params: {total_params:,} ({total_params * 4 / (1024 ** 2):.2f} MB)")
print(f"Trainable params: {trainable_params:,} ({trainable_params * 4 / (1024 ** 2):.2f} MB)")
print(f"Non-trainable params: {non_trainable_params:,} ({non_trainable_params * 4 / (1024 ** 2):.2f} B)")


### 7.2 - Define LOSO CV

In [ ]:
def LOSO_CV_fus(
  reference_df:pd.DataFrame=None,
  select_x_subj:list=None,
  data_types_to_include:list=None,
  freeze_base_models:bool=True,
  batch_size:int=30,
  num_epochs_fus:int=200,
  patience_fus:int=30,
  lambda_l2_fus:float=1e-3,
  lr_fus:float=1e-4,
  data_dir:str=None,
):
  # initilaize objects to store LOSO CV infos
  results = {}
  for data_type in data_types_to_include + ['fusion']:
    results[data_type] = {'dict_history':{}, 'dict_times':{}, 'dict_memory':{},
                          'dict_cm':{}, 'dict_clf_rep':{}, 'dict_acc':{}}

  sorted_subjects = sorted(reference_df['subject'].unique(), key=lambda x: int(x.replace('subject', '')))
  if isinstance(select_x_subj, list):
    sorted_subjects = [sorted_subjects[i-1] for i in select_x_subj]

  for subject in sorted_subjects:

    print(f'\n=== {subject.upper()} ===\n')

    # object to store models
    model_dict = {}

    # create train and validation reference data
    train_reference_df, valid_reference_df = reference_df[reference_df['subject']!=subject], reference_df[reference_df['subject']==subject]
    valid_reference_df = valid_reference_df[~valid_reference_df.index.str.contains('flipped')] # no augmented data for the validation set

    joints = [0,1,2,3,4,11,18,19,20,21,22,23,24,25]

    for data_type in data_types_to_include:

      if data_type == 'foot':
        train_steps = int(np.ceil(len(train_reference_df)/batch_size))
      else:
        chunk_counts = [count_chunks(fp, joints, clean_data=True, norm=True, target_size=50) for fp in train_reference_df.index]
        train_steps = int(np.ceil(sum(chunk_counts)/batch_size))
      valid_steps = int(np.ceil(len(valid_reference_df)/batch_size))

      # common args to create train and validation sets
      common_args = {'data_type':data_type,
                    'joints':joints,
                    'target_size':50,
                    'img_size':(128,48,1),
                    'clean_data':True,
                    'norm':True,
                    'center_ft':True,
                    'batch_size':batch_size}

      train_data = create_dataset(**common_args, reference_df=train_reference_df, crop_type='split_subsequence',
                                  shuffle=True, cache_file=f'train_{data_type}_{subject}')

      valid_data = create_dataset(**common_args, reference_df=valid_reference_df, crop_type='aggressive_center',
                                  shuffle=False, cache_file=f'valid_{data_type}_{subject}')

      # initialize classifier
      num_features = len(joints)*3 if data_type=='skeleton' else len(joints) if data_type=='speed' else 0
      model_type = 'gru' if data_type == 'skeleton' else ('gru' if data_type == 'speed' else 'cnn')
      # model_type = 'transformer' if data_type == 'skeleton' else ('gru' if data_type == 'speed' else 'cnn')

      lr_clf = 5e-5 if data_type=='foot' else 1e-4

      model_clf = define_model(data_type=data_type, model_type=model_type,
                               target_size=50, num_features=num_features,
                               img_size=(128,48,1), lambda_l2=5e-3,bidirectional=False)

      model_clf.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=lr_clf),
                        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                        metrics=['accuracy'])

      early_stop_callback_clf = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
      memory_callback_clf = MemoryLoggingCallback()

      start_time_clf = time.time()
      history_clf = model_clf.fit(train_data, epochs=200, steps_per_epoch=train_steps,
                                  validation_data=valid_data, validation_steps=valid_steps,
                                  callbacks=[early_stop_callback_clf, memory_callback_clf], verbose=0)

      delta_time_clf = time.time() - start_time_clf

      # save results
      cm, df_report, acc = compute_metrics_fus(model_clf, valid_reference_df, valid_data, valid_steps)

      results[data_type]['dict_times'][subject], results[data_type]['dict_history'][subject], results[data_type]['dict_memory'][subject] = delta_time_clf, history_clf, memory_callback_clf.memory_log
      results[data_type]['dict_cm'][subject], results[data_type]['dict_clf_rep'][subject], results[data_type]['dict_acc'][subject] = cm, df_report, acc

      # save model to create the fusion one
      model_dict[data_type] = model_clf

      del train_data, valid_data, model_clf

    ################
    # FUSION MODEL #
    ################
    chunk_counts = [count_chunks(fp, joints, clean_data=True, norm=True, target_size=50) for fp in train_reference_df.index]
    train_steps = int(np.ceil(sum(chunk_counts)/batch_size))

    # create dataset
    train_data_fus = create_fusion_dataset(reference_df=train_reference_df,
                                           data_types_to_include=data_types_to_include,
                                           shuffle=True, cache_file=f'train_fus_{subject}')

    valid_data_fus = create_fusion_dataset(reference_df=valid_reference_df, train_or_valid='valid',
                                           data_types_to_include=data_types_to_include,
                                           shuffle=False, cache_file=f'valid_fus_{subject}')

    # initialize and train model
    fus_model = build_fusion_model(model_dict, data_types_to_include, freeze_base_models, lambda_l2_fus)
    fus_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_fus),
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                      metrics=['accuracy'])

    early_stop_callback_fus = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience_fus, restore_best_weights=True)
    memory_callback_fus = MemoryLoggingCallback()

    start_time_fus = time.time()
    history_fus = fus_model.fit(train_data_fus, epochs=num_epochs_fus, steps_per_epoch=train_steps,
                                validation_data=valid_data_fus, validation_steps=valid_steps,
                                callbacks=[early_stop_callback_fus, memory_callback_fus], verbose=0)

    delta_time_fus = time.time() - start_time_fus

    # show and save results
    cm, df_report, acc = compute_metrics_fus(fus_model, valid_reference_df, valid_data_fus, valid_steps)

    results['fusion']['dict_times'][subject], results['fusion']['dict_history'][subject], results['fusion']['dict_memory'][subject] = delta_time_fus, history_fus, memory_callback_fus.memory_log
    results['fusion']['dict_cm'][subject], results['fusion']['dict_clf_rep'][subject], results['fusion']['dict_acc'][subject] = cm, df_report, acc

    print_time_info_fus(subject=subject, results=results, data_types_to_include=data_types_to_include)
    plot_history_fus(subject=subject, results=results, data_types_to_include=data_types_to_include)
    plot_metrics_fus(subject=subject, results=results, data_types_to_include=data_types_to_include)

    # remove cache and clear session
    for data_type in data_types_to_include + ['fus']:
      remove_cache_from_drive(cache_data=f'train_{data_type}_{subject}', data_dir=data_dir)
      remove_cache_from_drive(cache_data=f'valid_{data_type}_{subject}', data_dir=data_dir)

    tf.keras.backend.clear_session()
    gc.collect()

  return results


### 7.3 LOSO CV Fusion Model

#### 7.3.1 - Skeleton + Foot Pressure Data

In [ ]:
our_params = {
    'reference_df':loso_reference_df,
    'data_types_to_include':['skeleton', 'foot'],
    'freeze_base_models': False,
    'batch_size':30,
    'num_epochs_fus':200,
    'patience_fus':10,
    'lambda_l2_fus':5e-3,
    'lr_fus':5e-5,
    'data_dir':data_dir
}

results = LOSO_CV_fus(**our_params)


In [ ]:
print_overall_time(results, 'clf', 'fusion')


In [ ]:
print_overall_history(results, 'fusion')


In [ ]:
print_overall_epochs(results, 'clf', 'fusion')


In [ ]:
print_overall_cm(results, 'fusion')


In [ ]:
print_overall_clf_report(results, 'fusion')


In [ ]:
show_multiple_accuracies(results, ['fusion', 'skeleton', 'foot'])


In [ ]:
print_overall_max_gpu_memory_usage(results, 'clf', 'fusion')


#### 7.3.2 - Skeleton + Speed Data

In [ ]:
our_params = {
    'reference_df':loso_reference_df,
    'data_types_to_include':['skeleton', 'speed'],
    'freeze_base_models': False,
    'batch_size':30,
    'num_epochs_fus':200,
    'patience_fus':10,
    'lambda_l2_fus':5e-3,
    'lr_fus':5e-5,
    'data_dir':data_dir
}

results = LOSO_CV_fus(**our_params)


In [ ]:
print_overall_time(results, 'clf', 'fusion')


In [ ]:
print_overall_history(results, 'fusion')


In [ ]:
print_overall_epochs(results, 'clf', 'fusion')


In [ ]:
print_overall_cm(results, 'fusion')


In [ ]:
print_overall_clf_report(results, 'fusion')


In [ ]:
show_multiple_accuracies(results, ['fusion', 'skeleton', 'speed'])


In [ ]:
print_overall_max_gpu_memory_usage(results, 'clf', 'fusion')


#### 7.3.3 - Speed + Foot Pressure Data

In [ ]:
our_params = {
    'reference_df':loso_reference_df,
    'data_types_to_include':['speed', 'foot'],
    'freeze_base_models': False,
    'batch_size':30,
    'num_epochs_fus':200,
    'patience_fus':10,
    'lambda_l2_fus':5e-3,
    'lr_fus':5e-5,
    'data_dir':data_dir
}

results = LOSO_CV_fus(**our_params)


In [ ]:
print_overall_time(results, 'clf', 'fusion')


In [ ]:
print_overall_history(results, 'fusion')


In [ ]:
print_overall_epochs(results, 'clf', 'fusion')


In [ ]:
print_overall_cm(results, 'fusion')


In [ ]:
print_overall_clf_report(results, 'fusion')


In [ ]:
show_multiple_accuracies(results, ['fusion', 'speed', 'foot'])


In [ ]:
print_overall_max_gpu_memory_usage(results, 'clf', 'fusion')


#### 7.3.4 - Skeleton + Speed + Foot Pressure Data

In [ ]:
our_params = {
    'reference_df':loso_reference_df,
    'select_x_subj': [1,2,3,4,5,6],
    'data_types_to_include':['skeleton', 'speed', 'foot'],
    'freeze_base_models': False,
    'batch_size':30,
    'num_epochs_fus':200,
    'patience_fus':10,
    'lambda_l2_fus':5e-3,
    'lr_fus':5e-5,
    'data_dir':data_dir
}

results1 = LOSO_CV_fus(**our_params)


In [ ]:
# to save results
with open('FUSION_SK_SP_FT_LOSOa.pkl', 'wb') as f:
  pickle.dump(results1, f)


In [ ]:
our_params = {
    'reference_df':loso_reference_df,
    'select_x_subj': [7,8,9,10,11],
    'data_types_to_include':['skeleton', 'speed', 'foot'],
    'freeze_base_models': False,
    'batch_size':30,
    'num_epochs_fus':200,
    'patience_fus':10,
    'lambda_l2_fus':5e-3,
    'lr_fus':5e-5,
    'data_dir':data_dir
}

results2 = LOSO_CV_fus(**our_params)


In [ ]:
# to save results
with open('FUSION_SK_SP_FT_LOSOb.pkl', 'wb') as f:
  pickle.dump(results2, f)


In [ ]:
# to load results
with open('FUSION_SK_SP_FT_LOSOa.pkl', 'rb') as f:
  results1 = pickle.load(f)

with open('FUSION_SK_SP_FT_LOSOb.pkl', 'rb') as f:
  results2 = pickle.load(f)

results = merge_loso_results(results1, results2)


In [ ]:
print_overall_time(results, 'clf', 'fusion')


In [ ]:
print_overall_history(results, 'fusion')


In [ ]:
print_overall_epochs(results, 'clf', 'fusion')


In [ ]:
print_overall_cm(results, 'fusion')


In [ ]:
print_overall_clf_report(results, 'fusion')


In [ ]:
show_multiple_accuracies(results, ['fusion', 'skeleton', 'speed', 'foot'])


## 8 - Final Model Training and Evaluation

### 8.1 - Transformer Classifier LOSO CV with Test

In [ ]:
our_params = {'reference_df': loso_reference_df,
              'test_reference_df': test_reference_df,
              'model_type': 'transformer',
              'data_type': 'skeleton',
              'target_size': 50,
              'joints': _c.joints_to_include,
              'clean_data': True,
              'norm': True,
              'crop_type': 'split_subsequence',
              'batch_size': 30,
              'lambda_l2': 5e-3,
              'lr': 1e-4,
              'num_epochs_clf': 200,
              'patience_clf': 30,
              'data_dir':data_dir,
              }

results = LOSO_CV(**our_params)


In [ ]:
print_overall_time(results)


In [ ]:
print_overall_history(results)


In [ ]:
test_results = results['test_results']

In [ ]:
test_accuracies = []
test_precisions = []
test_recalls = []

for subj, report_df in test_results['clf_rep'].items():
    if isinstance(report_df, pd.DataFrame):
        precision = report_df.loc['macro avg', 'precision']
        recall = report_df.loc['macro avg', 'recall']
    else:
        precision = report_df['macro avg']['precision']
        recall = report_df['macro avg']['recall']

    accuracy = test_results['acc'][subj]

    test_accuracies.append(accuracy)
    test_precisions.append(precision)
    test_recalls.append(recall)

avg_test_accuracy = np.mean(test_accuracies)
avg_test_precision = np.mean(test_precisions)
avg_test_recall = np.mean(test_recalls)

print(f"\n==== AVERAGE TEST METRICS ====")
print(f"Accuracy: {avg_test_accuracy:.4f}")
print(f"Precision: {avg_test_precision:.4f}")
print(f"Recall: {avg_test_recall:.4f}")


In [ ]:
cm_dict = results['test_results']['cm']
conf_matrices = list(cm_dict.values())
total_cm = np.sum(conf_matrices, axis=0)
class_names = ['Normal', 'Antalgic', 'Lurching', 'Steppage', 'Stiff-legged', 'Trendelenburg']
plt.figure(figsize=(8, 6))
sns.heatmap(total_cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title("Test Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()


### 8.2 - Final Training with All Exemples

The final model will be trained on the same subjects used for LOSO CV, meaning all 11 subjects are used for training while the 12th remain as test set.
The model weights are then saved.

In [ ]:
joints_to_include=_c.joints_to_include
target_size=50

# Create Training Dataset
chunk_counts = [count_chunks(fp, joints_to_include, clean_data=True, norm=True, target_size=target_size) for fp in loso_reference_df.index]
train_steps = int(np.ceil(sum(chunk_counts)/30))

train_data = create_dataset(reference_df=loso_reference_df, data_type='skeleton', joints=joints_to_include,
                            target_size=target_size, clean_data=True, norm=True, crop_type='split_subsequence',
                            batch_size=30, shuffle=True, cache_file='final_train_data')

# Initialize the Model
model = Transformer_Skeleton(input_shape=(target_size, 3*len(joints_to_include)))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=30,restore_best_weights=True)

# Train the Model
history = model.fit(train_data, epochs=200, steps_per_epoch=train_steps, callbacks=[early_stopping], verbose=0)

# Save the Model
model.save('transformer_skeleton_model.h5')


In [ ]:
def plot_final_hist(history):
  fig, axs = plt.subplots(1, 2, figsize=(14, 6)) # do not use predefine fig size (in chapter 0)
  # loss
  axs[0].plot(history.history['loss'], label='Train loss')
  axs[0].set_xlabel('Epoch')
  axs[0].set_ylabel('Loss')
  axs[0].set_title(f'Loss vs Epoch')
  # accuracy
  axs[1].plot(history.history['accuracy'], label='Train accuracy')
  axs[1].set_xlabel('Epoch')
  axs[1].set_ylabel('Accuracy')
  axs[1].set_title(f'Accuracy vs Epoch')

  plt.tight_layout()
  plt.show()

plot_final_hist(history)


In [ ]:
# Train Data
train_data_eval = create_dataset(reference_df=loso_reference_df, data_type='skeleton', joints=joints_to_include,
                                 target_size=target_size, clean_data=True, norm=True, crop_type='aggressive_center',
                                 batch_size=30, shuffle=False, cache_file='final_train_data_eval')
train_steps_eval = int(np.ceil(len(loso_reference_df)/30))

cm_train, clf_report_train, acc_train = compute_metrics(model, loso_reference_df, train_data_eval, train_steps_eval, verbose=False)

# Test Data
test_reference_df = test_reference_df[~test_reference_df.index.str.contains('flipped')]
test_data_eval = create_dataset(reference_df=test_reference_df, data_type='skeleton', joints=joints_to_include,
                                target_size=target_size, clean_data=True, norm=True, crop_type='aggressive_center',
                                batch_size=30, shuffle=False, cache_file='final_test_data_eval')
test_steps_eval = int(np.ceil(len(test_reference_df)/30))

cm_test, clf_report_test, acc_test = compute_metrics(model, test_reference_df, test_data_eval, test_steps_eval, verbose=False)


In [ ]:
print("=== TRAIN SET ===\n")
print(f"Accuracy: {acc_train:.4f}\n")
print("Classification Report:\n", clf_report_train.to_string(), "\n")

print("\n\n=== TEST SET ===\n")
print(f"Accuracy: {acc_test:.4f}\n")
print("Classification Report:\n", clf_report_test.to_string(), "\n")

classes=list(_c.convrt_gait_dict.keys())
fig, axs = plt.subplots(1, 2, figsize=(14, 6)) # do not use predefine fig size (in chapter 0)

sns.heatmap(cm_train, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes, ax=axs[0])
axs[0].set_title('Train CM')
axs[0].set_xlabel('Predicted Label')
axs[0].set_ylabel('True Label')

sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes, ax=axs[1])
axs[1].set_title('Test CM')
axs[1].set_xlabel('Predicted Label')
axs[1].set_ylabel('True Label')

plt.tight_layout()
plt.show()


In [ ]:
remove_cache_from_drive(cache_data='final_train_data', data_dir=data_dir)
remove_cache_from_drive(cache_data='final_train_data_eval', data_dir=data_dir)
remove_cache_from_drive(cache_data='final_test_data_eval', data_dir=data_dir)
